In [ ]:
type Meat = Chicken | Beef | Pork | Fish | Tofu

type Ingredient = 
    | Cheese | Rice | Beans | Salsa | Guacamole | SourCream | Lettuce
    | Tomato | Onion | Cilantro | PicoDeGallo

type Burrito = Meat option * Ingredient list

let tortilla =
    Some (), []

let addMeat meat burrito =
    match burrito with
    | Some _, ingredients -> Some meat, ingredients
    | None, _ -> None, []

let addIngredient ingredient burrito =
    match burrito with
    | Some meat, ingredients -> Some meat, ingredient :: ingredients
    | None, _ -> None, []

let addMissionBurritoIngredients burrito =
    match burrito with
    | Some meat, ingredients -> Some meat, [Cheese; Rice; Beans] @ ingredients
    | None, _ -> None, []

let holdThe ingredient burrito =
    match burrito with
    | Some meat, ingredients -> Some meat, List.filter (fun i -> i <> ingredient) ingredients
    | None, _ -> None, []

let (>>=) burrito f =
    match burrito with
    | Some meat, ingredients -> f (Some meat, ingredients)
    | None, _ -> None, []

let burrito = 
  tortilla
  >>= addMeat Chicken 
  >>= addMissionBurritoIngredients
  >>= holdThe Cheese
  >>= addIngredient PicoDeGallo 
  >>= addIngredient Salsa
  >>= addIngredient Guacamole
  >>= addIngredient SourCream 

printfn "%A" burrito

In [ ]:
type Meat = Chicken | Beef | Pork | Fish | Veggie

type Ingredient = 
    | Cheese | Rice | Beans | Salsa | Guacamole | SourCream | Lettuce
    | Tomato | Onion | Cilantro | PicoDeGallo

type Burrito = Meat option * Ingredient list

let (>>=) burrito f =
    match burrito with
    | Some meat, ingredients -> f (Some meat, ingredients)
    | None, _ -> None, []

let returnBurrito (meat, ingredients) = meat, ingredients

let tortilla = returnBurrito (Some Veggie, [])

let addMeat meat (m, ingredients) = Some meat, ingredients

let addIngredient ingredient (meat, ingredients) =
    meat, ingredient :: ingredients

let addMissionBurritoIngredients (meat, ingredients) =
    meat, Cheese :: Rice :: Beans :: ingredients

let holdThe ingredient (meat, ingredients) =
    meat, List.filter (fun i -> i <> ingredient) ingredients

let burrito = 
    tortilla
    >>= addMeat Chicken
    >>= addMissionBurritoIngredients
    >>= holdThe Cheese
    >>= addIngredient PicoDeGallo
    >>= addIngredient Salsa
    >>= addIngredient Guacamole
    >>= addIngredient SourCream

printfn "%A" burrito

In [ ]:
type Meat = Chicken | Beef | Pork | Fish | Veggie
type Ingredient = Cheese | Rice | Beans | Salsa | Guacamole | SourCream | Lettuce | Tomato | Onion | Cilantro | PicoDeGallo

type BurritoState = { Meat: Meat option; Ingredients: Ingredient list }

type Burrito<'a> = Burrito of (BurritoState -> ('a * BurritoState))

type BurritoBuilder() =
    member _.Return(x) = Burrito(fun state -> (x, state))
    member _.Bind(Burrito(m), f) = 
        Burrito(fun state -> 
            let (a, newState) = m(state)
            let (Burrito n) = f(a)
            n(newState))
    member _.Zero() = Burrito(fun state -> ((), state))
    member this.Delay(f) = Burrito(fun state -> let (Burrito g) = f() in g(state))

    member this.AddMeat(meat) = 
        Burrito(fun state -> ((), { state with Meat = Some meat }))
    member this.AddIngredient(ingredient) = 
        Burrito(fun state -> ((), { state with Ingredients = ingredient :: state.Ingredients }))
    member this.RemoveIngredient(ingredient) = 
        Burrito(fun state -> ((), { state with Ingredients = List.filter ((<>) ingredient) state.Ingredients }))

let burrito = BurritoBuilder()

let buildBurrito = burrito {
    do! burrito.AddMeat Chicken
    do! burrito.AddIngredient Cheese
    do! burrito.AddIngredient Rice
    do! burrito.AddIngredient Beans
    do! burrito.RemoveIngredient Cheese  // Decided no cheese after all
    do! burrito.AddIngredient PicoDeGallo
    do! burrito.AddIngredient Salsa
    do! burrito.AddIngredient Guacamole
    do! burrito.AddIngredient SourCream
    return "Your burrito is ready!"
}

// Initial empty state of the burrito
let initialState = { Meat = None; Ingredients = [] }

// Running the computation
let (message, finalState) = let (Burrito f) = buildBurrito in f(initialState)

printfn "%s" message
printfn "%A" finalState

In [1]:
type Item = Sword | Shield | Potion | Gold of int
type Status = Alive | Dead

type Hero = {
    Name: string
    Inventory: Item list
    Health: int
    Status: Status
}

type Adventure = Hero option * string list

let startAdventure initialHero = 
  Some initialHero, ["The adventure begins!"]

let findItem item (hero, logs) =
    match hero with
    | Some h when h.Status = Alive -> Some { h with Inventory = item :: h.Inventory }, (item.ToString() + " found!") :: logs
    | _ -> None, "No hero to receive the item..." :: logs

let encounterEnemy damage (hero, logs) =
    match hero with
    | Some h when h.Health > damage -> 
        let newHealth = h.Health - damage
        let newStatus = if newHealth > 0 then Alive else Dead
        Some { h with Health = newHealth; Status = newStatus }, ("Encountered enemy, health now " + newHealth.ToString()) :: logs
    | Some h -> None, ("Hero fell in battle...") :: logs
    | None -> None, "No hero to fight the enemy..." :: logs

let (>>=) adventure f =
    match adventure with
    | Some hero, logs when hero.Status = Alive -> f (Some hero, logs)
    | _ -> adventure  // Return the current state without further processing


let completeAdventure = 
    startAdventure {
        Name = "Arthur"
        Inventory = []
        Health = 100
        Status = Alive
    }
    >>= findItem (Gold 50)
    >>= findItem Potion
    >>= encounterEnemy 20
    >>= findItem Sword
    >>= encounterEnemy 50

match completeAdventure with
| Some hero, logs -> 
    printfn "%A" hero
    printfn "Logs: %s" (String.Join("\n", List.rev logs))
| None, logs -> 
    printfn "Logs: %s" (String.Join("\n", List.rev logs))


{ Name = "Arthur"
  Inventory = [Sword; Potion; Gold 50]
  Health = 30
  Status = Alive }
Logs: The adventure begins!
Gold 50 found!
Potion found!
Encountered enemy, health now 80
Sword found!
Encountered enemy, health now 30
